# GTSF phase I: biomass prediction

In this notebook, we are forecasting the weights by finding the closest blender model

### Look at the volumes created with blender

Load blender data

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
import json
import cv2

import glob
import os
import boto3
import tempfile
from sqlalchemy import create_engine, MetaData, Table, select, and_, func
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.automap import automap_base
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from scipy.stats import norm
from scipy.linalg import cholesky
import tqdm
import pickle
from itertools import combinations
from aquabyte.data_access_utils import DataAccessUtils
from aquabyte.optics import convert_to_world_point, depth_from_disp, pixel2world, euclidean_distance

from PIL import Image, ImageDraw
from multiprocessing import Pool, Manager
import copy
import uuid
from sklearn.preprocessing import StandardScaler

pd.set_option('display.max_rows', 500)





In [ ]:
data_access_utils = DataAccessUtils('/root/data/')

<h1> Get world keypoint coordinates from GTSF data </h1>

In [ ]:
aws_credentials = json.load(open(os.environ["AWS_CREDENTIALS"]))
s3_client = boto3.client('s3', aws_access_key_id=aws_credentials["aws_access_key_id"],
                         aws_secret_access_key=aws_credentials["aws_secret_access_key"],
                         region_name="eu-west-1")


sql_credentials = json.load(open(os.environ["SQL_CREDENTIALS"]))
sql_engine = create_engine("postgresql://{}:{}@{}:{}/{}".format(sql_credentials["user"], sql_credentials["password"],
                           sql_credentials["host"], sql_credentials["port"],
                           sql_credentials["database"]))

Session = sessionmaker(bind=sql_engine)
session = Session()

Base = automap_base()
Base.prepare(sql_engine, reflect=True)
Enclosure = Base.classes.enclosures
Calibration = Base.classes.calibrations
GtsfDataCollection = Base.classes.gtsf_data_collections
StereoFramePair = Base.classes.stereo_frame_pairs


<h1> Load GTSF dataset </h1>

In [ ]:
df = pd.read_hdf('/root/data/df_cache.h5')
df = df[df.project_name != 'Automated keypoints detection']

In [ ]:
df.project_name.unique()

<h1> Define biomass prediction method </h1>

In [ ]:
def coord2biomass_linear(world_keypoints, model):
    """from coordinates to biomass"""

    mean = model['mean']
    std= model['std']
    PCA_components = model['PCA_components']
    reg_coef = model['reg_coef']
    reg_intercept = model['reg_intercept']
    body_parts = model['body_parts']
    # calculate pairwise distances for production coord
    # based on the exact ordering reflected in the body_parts
    # variable above

    pairwise_distances = []
    for i in range(len(body_parts)-1):
        for j in range(i+1, len(body_parts)):
            dist = euclidean_distance(world_keypoints[body_parts[i]], world_keypoints[body_parts[j]])
            pairwise_distances.append(dist)

    interaction_values_quadratic = []
    for i in range(len(pairwise_distances)):
        for j in range(i, len(pairwise_distances)):
            dist1 = pairwise_distances[i]
            dist2 = pairwise_distances[j]
            interaction_values_quadratic.append(dist1 * dist2)

    interaction_values_cubic = []
    for i in range(len(pairwise_distances)):
        for j in range(i, len(pairwise_distances)):
            for k in range(j, len(pairwise_distances)):
                dist1 = pairwise_distances[i]
                dist2 = pairwise_distances[j]
                dist3 = pairwise_distances[k]
                interaction_values_cubic.append(dist1 * dist2 * dist3)


    X = np.array(pairwise_distances + interaction_values_quadratic + interaction_values_cubic)

    X_normalized = (X - model['mean']) / model['std']
    X_transformed = np.dot(X_normalized, model['PCA_components'].T)
    prediction = np.dot(X_transformed, reg_coef) + reg_intercept
    return prediction

<h1> Define Stereo Parameters by Project Type </h1>

In [ ]:
# pre-axiom parameters

parameters_by_project_type = {}
gtsf_data_collection = session.query(GtsfDataCollection).first()
calibration = session.query(Calibration) \
             .filter(Calibration.enclosure_id == gtsf_data_collection.enclosure_id) \
             .order_by(Calibration.utc_timestamp.desc()) \
             .first()

enclosure = session.query(Enclosure).get(calibration.enclosure_id)


focal_length = float(calibration.predicted_focal_length_mm) / (1e3)
baseline = float(calibration.predicted_baseline_mm) / (1e3)
pixel_size_m = float(enclosure.pixel_width_um) / (1e6)
focal_length_pixel = focal_length / pixel_size_m
image_sensor_width = float(enclosure.sensor_width_mm) / (1e3)
image_sensor_height = float(enclosure.sensor_height_mm) / (1e3)
pixel_count_width = enclosure.image_num_pixels_width
pixel_count_height = enclosure.image_num_pixels_height

parameters_pre_axiom = {
    'baseline': baseline,
    'focalLengthPixel': focal_length_pixel,
    'imageSensorWidth': image_sensor_width,
    'imageSensorHeight': image_sensor_height,
    'pixelCountWidth': pixel_count_width,
    'pixelCountHeight': pixel_count_height,
    'pixelSize': pixel_size_m
}

parameters_pre_axiom['focalLength'] = parameters_pre_axiom['focalLengthPixel'] * parameters_pre_axiom['pixelSize']
parameters_by_project_type['pre-axiom'] = parameters_pre_axiom


In [ ]:
# post-axiom parameters

stereo_params = json.load(open('/root/data/s3/aquabyte-stereo-parameters/L40020185_R40020187/latest_L40020185_R40020187_stereo-parameters.json'))
focal_length_pixel = stereo_params['CameraParameters1']['FocalLength'][0]
baseline = abs(stereo_params['TranslationOfCamera2'][0] / 1e3) # convert millimeters to meters and use absolute value
pixel_size_m = 3.45 * 1e-6
focal_length = focal_length_pixel * pixel_size_m
image_sensor_width = 0.01412
image_sensor_height = 0.01035
pixel_count_width = 4096
pixel_count_height = 3000

parameters_post_axiom = {
    'baseline': baseline,
    'focalLengthPixel': focal_length_pixel,
    'focalLength': focal_length,
    'imageSensorWidth': image_sensor_width,
    'imageSensorHeight': image_sensor_height,
    'pixelCountWidth': pixel_count_width,
    'pixelCountHeight': pixel_count_height,
    'pixelSize': pixel_size_m
}

parameters_post_axiom['focalLength'] = parameters_post_axiom['focalLengthPixel'] * parameters_post_axiom['pixelSize']
parameters_by_project_type['post-axiom'] = parameters_post_axiom


<h1> Construct analysis dataframe </h1>

In [ ]:
# initialize analysis df
analysis_df = pd.DataFrame()

# define baseline biomass model and body_parts
model = pickle.load(open('/root/data/alok/biomass_estimation/models/20190716_model_4_eig.pkl', 'rb'))
body_parts = sorted([
    'TAIL_NOTCH',
    'ADIPOSE_FIN',
    'ANAL_FIN',
    'PECTORAL_FIN',
    'PELVIC_FIN',
    'DORSAL_FIN',
    'UPPER_LIP',
    'EYE'
])


# define jitter values & number of trials per jitter, and initialize random seed
jitter_values_x = [0, 2, 10, 20, 30, 40, 50]
np.random.seed(0)
trials = 10


# Correlation matrix
corr_mat= np.array([[1.0, 0.8],
                    [0.8, 1.0]])

# Compute the (upper) Cholesky decomposition matrix
upper_chol = cholesky(corr_mat)


for idx, row in df.iterrows():
    if idx % 10 == 0:
        print(idx)
    
    
    left_keypoints_original = row.left_keypoints
    right_keypoints_original = row.right_keypoints
    project_name = row.project_name
    if 'GTSF Phase I Keypoint Annotations' in project_name:
        project_type = 'pre-axiom'
    elif project_name == 'Underwater Live GTSF - Axiom Calibration Full':
        project_type = 'post-axiom'
    
    
    
    # introduce small jitter
    for jitter_value_x in jitter_values_x:
        T = 1 if jitter_value_x == 0 else trials
        for t in range(T):
            
            # Generate 3 series of normally distributed (Gaussian) numbers
            rnd = np.random.normal(0.0, jitter_value_x, size=2)
            jitters = rnd @ upper_chol
            
            left_keypoints = {bp: [left_keypoints_original[bp][0] + jitters[0],
                                   left_keypoints_original[bp][1]] for bp in body_parts}
            right_keypoints = {bp: [right_keypoints_original[bp][0] + jitters[1],
                                    right_keypoints_original[bp][1]] for bp in body_parts}

            world_keypoints = {}
            
            for bp in body_parts:
                lkp, rkp = left_keypoints[bp], right_keypoints[bp]
                d = abs(lkp[0] - rkp[0])

                # compute world key point
                depth = depth_from_disp(d, parameters_by_project_type[project_type])
                wkp = convert_to_world_point(lkp[0], lkp[1], depth, parameters_by_project_type[project_type])
                
                world_keypoints[bp] = wkp

            predicted_weight_linear = coord2biomass_linear(world_keypoints, model)

            df_row = {
                'gtsf_fish_identifier': row.gtsf_fish_identifier,
                'predicted_weight_linear': predicted_weight_linear,
                'weight': row.weight,
                'trial': t,
                'jitter_value_x': jitter_value_x,
                'stereo_frame_pair_id': row.stereo_frame_pair_id
            }
            
            analysis_df = analysis_df.append(df_row, ignore_index=True)





<h1> Plot precictions vs. ground truth for different jitter values </h1>

In [ ]:
m = (analysis_df.stereo_frame_pair_id != 2674) & (analysis_df.stereo_frame_pair_id != 1972)
for jitter_value_x in jitter_values_x:
    mask = (analysis_df.jitter_value_x == jitter_value_x) & m
    average_prediction_error = (analysis_df[mask].predicted_weight_linear.mean() - analysis_df[mask].weight.mean())/analysis_df[mask].weight.mean()
    print('Average prediction error: {}'.format(average_prediction_error))
    plt.figure(figsize=(20, 10))
    plt.scatter(analysis_df[mask].weight, analysis_df[mask].predicted_weight_linear)
    plt.title('Mean Jitter along x-axis = {} pixels'.format(jitter_value_x))
    plt.xlabel('Predicted weight (grams)')
    plt.ylabel('Ground truth weight (grams)')
    plt.plot([0, 10000], [0, 10000], color='red')
    plt.grid()
    plt.show()

<h1> Per-fish accuracy & precision metrics </h1>

In [ ]:
results_df = pd.DataFrame()
for jitter_value_x in jitter_values_x:
    for stereo_frame_pair_id in sorted(analysis_df.stereo_frame_pair_id.unique()):
        if (stereo_frame_pair_id == 2674) or (stereo_frame_pair_id == 1972):
            continue
        mask = (analysis_df.jitter_value_x == jitter_value_x) & (analysis_df.stereo_frame_pair_id == stereo_frame_pair_id)
        error_mean = (analysis_df[mask].predicted_weight_linear.mean() - analysis_df[mask].weight.mean())
        error_std = analysis_df[mask].predicted_weight_linear.std()
        error_mean_pct = error_mean / analysis_df[mask].weight.mean()
        error_std_pct = error_std / analysis_df[mask].weight.mean()
        row = {}
        row['jitter_value_x'] = jitter_value_x
        row['stereo_frame_pair_id'] = stereo_frame_pair_id
        row['error_mean'] = error_mean
        row['error_std'] = error_std
        row['error_mean_pct'] = error_mean_pct
        row['error_std_pct'] = error_std_pct
        results_df = results_df.append(row, ignore_index=True)
        
        

In [ ]:
for jitter_value_x in jitter_values_x:
    jitter_mask = results_df.jitter_value_x == jitter_value_x
    avg_weight = analysis_df.weight.mean()
    avg_error_mean = results_df[jitter_mask].error_mean.mean()
    avg_error_std = results_df[jitter_mask].error_std.mean()
    avg_error_mean_pct = avg_error_mean / avg_weight
    avg_error_std_pct = avg_error_std / avg_weight
    print('Jitter value: {}'.format(jitter_value_x))
    print('=' * 50)
    print('Average prediction error (grams): {}'.format(avg_error_mean))
    print('Average prediction spread (grams): {}'.format(avg_error_std))
    print('Average prediction error (percentage): {}'.format(avg_error_mean_pct))
    print('Average prediction spread (percentage): {}'.format(avg_error_std_pct))